In [1]:
import openai

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from pydantic import BaseModel, Field
from typing import List
from anthropic import Anthropic
import os
import json

In [5]:
class TextAnalysis(BaseModel):
    sentiment: str = Field(description="Overall sentiment of the text (positive, negative, or neutral)")
    main_topics: List[str] = Field(description="List of main topics in the text")
    word_count: int = Field(description="Total word count of the text")

In [14]:
def analyze_text_with_claude(api_key: str, text: str) -> TextAnalysis:
    client = Anthropic(api_key=api_key)
 
    text_analysis_schema = TextAnalysis.model_json_schema()
 
    tools = [
        {
            "name": "build_text_analysis_result",
            "description": "build the text analysis object",
            "input_schema": text_analysis_schema
        }
    ]
 
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1200,
        temperature=0.2,
        system="You are analyzing the sentiment of a text.",
        messages=[
            {
                "role": "user",
                "content": f"{text}"
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "build_text_analysis_result"}
    )
 
    function_call = message.content[0].input
    return TextAnalysis(**function_call)

# test it out!
api_key = os.environ.get("ANTHROPIC_API_KEY")
sample_text = "Claude tool use can be quite useful if used correctly."
 
print(f"Text: {sample_text}\nAnalyzing text...\n")
analysis = analyze_text_with_claude(api_key, sample_text)
print(f"Sentiment: {analysis.sentiment}")
print(f"Main topics: {', '.join(analysis.main_topics)}")
print(f"Word count: {analysis.word_count}")

Text: Claude tool use can be quite useful if used correctly.
Analyzing text...

Sentiment: positive
Main topics: tool use, Claude
Word count: 10


In [8]:
def list_anthropic_models(api_key: str) -> List[str]:
    client = Anthropic(api_key=api_key)
    models = client.models.list()
    return [model.id for model in models]  # Use id instead of name

# Test it out
models = list_anthropic_models(os.getenv("ANTHROPIC_API_KEY"))
print("Available Anthropic models:")
for model in models:
    print(f"- {model}")


Available Anthropic models:
- claude-3-5-sonnet-20241022
- claude-3-5-haiku-20241022
- claude-3-5-sonnet-20240620
- claude-3-haiku-20240307
- claude-3-opus-20240229
- claude-3-sonnet-20240229
- claude-2.1
- claude-2.0


In [22]:
# First, let's add our Pydantic models
class Step(BaseModel):
    action: str
    inputs: List[str]
    outputs: List[str]
    constraints: List[str] = Field(default_factory=list)

class ActionPattern(BaseModel):
    type: str
    frequency: str
    steps: List[Step]

class CurrentState(BaseModel):
    resources: dict = Field(default_factory=dict)
    constraints: dict = Field(default_factory=dict)
    progress_metrics: dict = Field(default_factory=dict)

class ExecutionStrategy(BaseModel):
    cycle: str
    checkpoints: List[str]
    memory_requirements: List[str]

class TaskPlan(BaseModel):
    goal: str
    success_metric: str
    current_state: CurrentState
    execution_strategy: ExecutionStrategy
    action_patterns: List[ActionPattern]

In [24]:

task = "Research and compile healthy vegetarian dinner recipes for a week"
print(f"\nTask: {task}")
print("-" * 50)
plan = generate_task_plan(os.getenv("ANTHROPIC_API_KEY"), task)
print(json.dumps(plan.model_dump(), indent=2))


Task: Research and compile healthy vegetarian dinner recipes for a week
--------------------------------------------------
{
  "goal": "Research and compile healthy vegetarian dinner recipes for a week",
  "success_metric": "Have a set of 7 healthy vegetarian dinner recipes to cook for the week",
  "current_state": {
    "resources": {
      "time_available": "1 week",
      "cooking_experience": "intermediate"
    },
    "constraints": {
      "dietary_restrictions": "vegetarian"
    },
    "progress_metrics": {
      "number_of_recipes_found": 0
    }
  },
  "execution_strategy": {
    "cycle": "daily",
    "checkpoints": [
      "Identify recipe sources",
      "Compile list of potential recipes",
      "Evaluate recipes for health and feasibility",
      "Select 7 recipes"
    ],
    "memory_requirements": [
      "list of recipe sources",
      "list of potential recipes",
      "evaluation criteria for recipes"
    ]
  },
  "action_patterns": [
    {
      "type": "Research",
  

In this notebook, we're going to build an agentic agent that can do tasks by itself.

How to use tools?

Extract tool input(s), run code, and return results: (API request)

On the client side, you should extract the tool name and input(s) from Claude's tool use request.
Run the actual tool code on the client side.
Return the results to Claude by continuing the conversation with a new user message containing a tool_result content block.


Hm.. do we want to have threads be a composite of Message and json types?

```
Message(id='msg_018XHYaQRDBURUvsCTY5wyRd', content=[TextBlock(text='Here is how we can check the amount of memory available in the current environment:', type='text'), ToolUseBlock(id='toolu_01Q8sFHK3EnUoohV1vfzHNf6', input={'code': 'import psutil\n\ntotal_memory = psutil.virtual_memory().total\nprint(f"Total memory: {total_memory / (1024 ** 2):.2f} MB")'}, name='run_python_code', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=439, output_tokens=116))
```

In [4]:

class FakeDatabase:
    def __init__(self):
        self.customers = [
            {
                "id": "1213210",
                "name": "John Doe",
                "email": "john@gmail.com",
                "phone": "123-456-7890",
                "username": "johndoe",
            },
            {
                "id": "2837622",
                "name": "Priya Patel",
                "email": "priya@candy.com",
                "phone": "987-654-3210",
                "username": "priya123",
            },
            {
                "id": "3924156",
                "name": "Liam Nguyen",
                "email": "lnguyen@yahoo.com",
                "phone": "555-123-4567",
                "username": "liamn",
            },
            {
                "id": "4782901",
                "name": "Aaliyah Davis",
                "email": "aaliyahd@hotmail.com",
                "phone": "111-222-3333",
                "username": "adavis",
            },
            {
                "id": "5190753",
                "name": "Hiroshi Nakamura",
                "email": "hiroshi@gmail.com",
                "phone": "444-555-6666",
                "username": "hiroshin",
            },
            {
                "id": "6824095",
                "name": "Fatima Ahmed",
                "email": "fatimaa@outlook.com",
                "phone": "777-888-9999",
                "username": "fatimaahmed",
            },
            {
                "id": "7135680",
                "name": "Alejandro Rodriguez",
                "email": "arodriguez@protonmail.com",
                "phone": "222-333-4444",
                "username": "alexr",
            },
            {
                "id": "8259147",
                "name": "Megan Anderson",
                "email": "megana@gmail.com",
                "phone": "666-777-8888",
                "username": "manderson",
            },
            {
                "id": "9603481",
                "name": "Kwame Osei",
                "email": "kwameo@yahoo.com",
                "phone": "999-000-1111",
                "username": "kwameo",
            },
            {
                "id": "1057426",
                "name": "Mei Lin",
                "email": "meilin@gmail.com",
                "phone": "333-444-5555",
                "username": "mlin",
            },
        ]

        self.orders = [
            {
                "id": "24601",
                "customer_id": "1213210",
                "product": "Wireless Headphones",
                "quantity": 1,
                "price": 79.99,
                "status": "Shipped",
            },
            {
                "id": "13579",
                "customer_id": "1213210",
                "product": "Smartphone Case",
                "quantity": 2,
                "price": 19.99,
                "status": "Processing",
            },
            {
                "id": "97531",
                "customer_id": "2837622",
                "product": "Bluetooth Speaker",
                "quantity": 1,
                "price": "49.99",
                "status": "Shipped",
            },
            {
                "id": "86420",
                "customer_id": "3924156",
                "product": "Fitness Tracker",
                "quantity": 1,
                "price": 129.99,
                "status": "Delivered",
            },
            {
                "id": "54321",
                "customer_id": "4782901",
                "product": "Laptop Sleeve",
                "quantity": 3,
                "price": 24.99,
                "status": "Shipped",
            },
            {
                "id": "19283",
                "customer_id": "5190753",
                "product": "Wireless Mouse",
                "quantity": 1,
                "price": 34.99,
                "status": "Processing",
            },
            {
                "id": "74651",
                "customer_id": "6824095",
                "product": "Gaming Keyboard",
                "quantity": 1,
                "price": 89.99,
                "status": "Delivered",
            },
            {
                "id": "30298",
                "customer_id": "7135680",
                "product": "Portable Charger",
                "quantity": 2,
                "price": 29.99,
                "status": "Shipped",
            },
            {
                "id": "47652",
                "customer_id": "8259147",
                "product": "Smartwatch",
                "quantity": 1,
                "price": 199.99,
                "status": "Processing",
            },
            {
                "id": "61984",
                "customer_id": "9603481",
                "product": "Noise-Cancelling Headphones",
                "quantity": 1,
                "price": 149.99,
                "status": "Shipped",
            },
            {
                "id": "58243",
                "customer_id": "1057426",
                "product": "Wireless Earbuds",
                "quantity": 2,
                "price": 99.99,
                "status": "Delivered",
            },
            {
                "id": "90357",
                "customer_id": "1213210",
                "product": "Smartphone Case",
                "quantity": 1,
                "price": 19.99,
                "status": "Shipped",
            },
            {
                "id": "28164",
                "customer_id": "2837622",
                "product": "Wireless Headphones",
                "quantity": 2,
                "price": 79.99,
                "status": "Processing",
            },
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")

        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None

    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"


from standardbackend.tools.base import Tool
from pydantic import BaseModel


class GetUserInput(BaseModel):
    key: str
    value: str


class GetOrderInput(BaseModel):
    order_id: str


class CancelOrderInput(BaseModel):
    order_id: str


class GetCustomerOrdersInput(BaseModel):
    customer_id: str


db = FakeDatabase()


def fail_tool(input):
    raise Exception("Not implemented")


tools = [
    Tool(
        name="get_user",
        description="Get a user by their email, phone, or username",
        input_schema=GetUserInput,
        execute=lambda input: db.get_user(input.key, input.value),
    ),
    Tool(
        name="get_order",
        description="Get an order by its ID",
        input_schema=GetOrderInput,
        execute=lambda input: db.get_order_by_id(input.order_id),
    ),
    Tool(
        name="cancel_order",
        description="Cancel an order by its ID",
        input_schema=CancelOrderInput,
        execute=lambda input: db.cancel_order(input.order_id),
    ),
    Tool(
        name="get_customer_orders",
        description="Get all orders for a customer by their ID",
        input_schema=GetCustomerOrdersInput,
        execute=lambda input: db.get_customer_orders(input.customer_id),
    ),
]

In [17]:
# Reload the thread module to get latest changes
from importlib import reload
import standardbackend
reload(standardbackend)
from standardbackend import Thread, Agent, pretty_print_messages

a = Agent(name="Sam", prompt="You are a helpful assistant that can answer questions and help with tasks.")
t = Thread(agent=a, tools=tools)
t.send_message("hey, what orders do I have? My email is john@gmail.com")
pretty_print_messages(t.messages)


[User]
hey, what orders do I have? My email is john@gmail.com

[Assistant]
Okay, let me look up your orders using your email address.

[Tool Use]
Tool: get_user
Input: {'key': 'email', 'value': 'john@gmail.com'}

[User]

[Tool Result]
{'id': '1213210', 'name': 'John Doe', 'email': 'john@gmail.com', 'phone': '123-456-7890', 'username': 'johndoe'}

[Assistant]
Okay, I found your user profile. Now let me retrieve all the orders associated with your customer ID.

[Tool Use]
Tool: get_customer_orders
Input: {'customer_id': '1213210'}

[User]

[Tool Result]
[{'id': '24601', 'customer_id': '1213210', 'product': 'Wireless Headphones', 'quantity': 1, 'price': 79.99, 'status': 'Shipped'}, {'id': '13579', 'customer_id': '1213210', 'product': 'Smartphone Case', 'quantity': 2, 'price': 19.99, 'status': 'Processing'}, {'id': '90357', 'customer_id': '1213210', 'product': 'Smartphone Case', 'quantity': 1, 'price': 19.99, 'status': 'Shipped'}]

[Assistant]
Based on the results, it looks like you have 

In [5]:
from standardbackend.tools.base import Tool
from standardbackend import Thread, Agent, pretty_print_messages
from pydantic import BaseModel, Field
from typing import List

# First, let's add our Pydantic models
class Step(BaseModel):
    action: str
    inputs: List[str]
    outputs: List[str]
    constraints: List[str] = Field(default_factory=list)

class ActionPattern(BaseModel):
    type: str
    frequency: str
    steps: List[Step]

class CurrentState(BaseModel):
    resources: dict = Field(default_factory=dict)
    constraints: dict = Field(default_factory=dict)
    progress_metrics: dict = Field(default_factory=dict)

class ExecutionStrategy(BaseModel):
    cycle: str
    checkpoints: List[str]
    memory_requirements: List[str]

class TaskPlan(BaseModel):
    goal: str
    success_metric: str
    current_state: CurrentState
    execution_strategy: ExecutionStrategy
    action_patterns: List[ActionPattern]

# Define a tool that takes in a task description and returns a task plan
tools = [
    Tool(
        name="generate_task_plan",
        description="Generate a detailed execution plan for a task",
        input_schema=TaskPlan,
        execute=lambda x: "Success! You can print the results as-is",
    )
]

# Finally, allow our agent to use this tool
planning_agent = Agent(name="Planning Agent", prompt="You are a task planning expert who breaks down complex tasks into detailed, actionable steps.")
planning_thread = Thread(agent=planning_agent, tools=tools)
planning_thread.send_message("Create a detailed execution plan for the following task: Research and compile healthy vegetarian dinner recipes for a week")
pretty_print_messages(planning_thread.messages)


[User]
Create a detailed execution plan for the following task: Research and compile healthy vegetarian dinner recipes for a week

[Assistant]
Okay, let's generate a detailed execution plan for the task of researching and compiling healthy vegetarian dinner recipes for a week:

[Tool Use]
Tool: generate_task_plan
Input: {'goal': 'Research and compile healthy vegetarian dinner recipes for a week', 'success_metric': 'Compile a set of 7 healthy, varied vegetarian dinner recipes', 'current_state': {'constraints': {'dietary_restrictions': ['vegetarian'], 'time_available': '1 week'}, 'progress_metrics': {'recipes_compiled': 0}, 'resources': {'cooking_experience': 'intermediate', 'access_to_grocery_store': True}}, 'execution_strategy': {'cycle': 'daily', 'checkpoints': ['3 recipes compiled', '5 recipes compiled', '7 recipes compiled'], 'memory_requirements': ['recipe database', 'nutrition information']}, 'action_patterns': [{'type': 'research', 'frequency': 'daily', 'steps': [{'action': 'sea